In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

OUTDIR = 'data'


/home/kirell/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [3]:
# LOCAL imports
import rts
import rts.pipeline
import rts.utils
import rts.io.media
import rts.features.audio
import rts.features.text

LOG = rts.utils.get_logger()

/home/kirell/.cache/pypoetry/virtualenvs/rts-506MfBOj-py3.10/lib/python3.10/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
# vidx = rts.metadata.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.metadata.read_all_metadata(METADATA, vidx)
# rts.metadata.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

df = rts.metadata.load_metadata_hdf5(METADATA, 'rts_metadata')
sample_df = rts.metadata.get_one_percent_sample(df)
# rts.metadata.export_metadata_stats(sample_df, Path(OUTDIR) / 'sample')
# rts.metadata.export_metadata_stats(df, OUTDIR)
sample_df.head()

,guid,mediaFolderPath,mediaDuration,ratio,formatResolution,publishedDate,categoryName,assetType,contentType,backgoundType,collection,publishedBy,rights,title,resume,geoTheme,resumeSequence,created,published,sequences
mediaId,,,,,,,,,,,,,,,,,,,,
ZE002100,AA1101000770,/mnt/rts/0/0/1/ZE002100,1524,16:9,SD,2010-04-29T17:40:00Z,Programme,Programme,Emissions de conseil,None,Dolce vita,TSR 1,Restriction/Condition,CAMPING - 10.04.29,"* Plateau : 20100429, SIKI Muriel reçoit ses i...","[LIEU INDÉTERMINÉ, VAUD, PLATEAU]",[00:01.24\r\nPlateau : diverses présentations ...,1670505072,1272562800,"{'890511': {'guid': 'AA1101000770', 'mediaId':..."
ZS009400,AA1106004840,/mnt/rts/0/0/4/ZS009400,530,16:9,SD,2011-03-05T00:00:00Z,Programme,Programme,Autres émissions religieuses,None,Faut pas croire,TSR 1,Restriction/Condition,11.03.05,"* Plateau : 20110305, BOISSET Christophe reçoi...","[LIEU INDÉTERMINÉ, NEUVEVILLE, LA, PLATEAU]",[00:01.18\r\nPlateau : diverses présentations ...,1670505072,1299283200,"{'897154': {'guid': 'AA1106004840', 'mediaId':..."
ZM001400,AA1201021870,/mnt/rts/0/0/4/ZM001400,43,16:9,SD,2012-01-22T00:00:00Z,Programme,Programme,"Bandes-annonces, lancements",None,Bande Annonce,TSR 2,Inconnu,Alain Delon. 2BLS CINEMA 2201,None,None,None,1670505073,1327190400,{}
ZT002200,AA1202023530,/mnt/rts/0/0/2/ZT002200,4445,16:9,SD,2012-04-08T00:00:00Z,Programme,Retransmission sportive,Retransmissions en direct,None,Motocyclisme,TSR 2,Restriction/Condition,MOTO3 - GRAND PRIX DU QATAR LOSAIL 2012,None,None,None,1670505073,1333843200,{}
ZM002200,AA1203026020,/mnt/rts/0/0/2/ZM002200,60,16:9,HD,2012-04-07T00:00:00Z,Programme,Programme,"Bandes-annonces, lancements",None,Bande Annonce,None,Inconnu,BLS SAMEDI 0704,None,None,None,1670505073,1333756800,{}


In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)

In [ ]:
# Fulltext Index from HDF5
# midx.delete_all_documents()
rts.db.fulltext.index_media(df, midx)
# Wait 30 sec for full indexation
midx.get_stats().number_of_documents  # 522103

In [6]:
# Extract location from transcripts and build dataframe
ts = rts.pipeline.load_all_transcripts(LOCAL_VIDEOS)
fts = rts.features.text.build_location_df(ts)
fts = rts.metadata.merge_location_df_with_metadata(sample_df, fts)
fts.head()

,location,s,e,t,mediaFolderPath,publishedDate,title,collection,contentType
mediaId,,,,,,,,,
ZB080800,Genève,76.18,78.46,"Tant à Berne qu'à Genève,",/mnt/rts/0/0/8/ZB080800,1990-06-28T00:00:00Z,90.06.28,Télé journal,None
ZB080800,Bern,76.18,78.46,"Tant à Berne qu'à Genève,",/mnt/rts/0/0/8/ZB080800,1990-06-28T00:00:00Z,90.06.28,Télé journal,None
ZB066800,Genève,156.00,159.00,la voix de Genève et de la Suisse du sud.,/mnt/rts/0/0/8/ZB066800,1999-03-01T00:00:00Z,PETER GOLDMARK PAR FRANK PEEL - 99.03.01 - IN ...,Les grands entretiens,"Rencontres, entretiens, portraits"
ZB066800,Genève,284.00,286.00,"ou beaucoup d'organisations basées sur la Genève,",/mnt/rts/0/0/8/ZB066800,1999-03-01T00:00:00Z,PETER GOLDMARK PAR FRANK PEEL - 99.03.01 - IN ...,Les grands entretiens,"Rencontres, entretiens, portraits"
ZB066800,Genève,1276.00,1277.00,de Genève.,/mnt/rts/0/0/8/ZB066800,1999-03-01T00:00:00Z,PETER GOLDMARK PAR FRANK PEEL - 99.03.01 - IN ...,Les grands entretiens,"Rencontres, entretiens, portraits"


In [ ]:
import PIL
import os
from PIL import ImageOps, Image

In [ ]:
im = Image.open(os.path.join(OUTDIR, 'test_image.jpg'))
rts.io.media.save_image_pyramid(im, Path(OUTDIR) / 'test_image_pyramid', 'test_image', True)

In [7]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')
# rts.io.media.get_media_info(video_path)

# scenes = rts.io.media.detect_scenes(video_path)
# scenes = rts.io.media.filter_scenes(scenes, 5)
# clips = rts.io.media.save_scenes_images(scenes, video_path, OUTDIR)
# rts.pipeline.save_scenes(clips, OUTDIR)

In [5]:
transcript = rts.pipeline.transcribe_media(
    OUTDIR,
    None, 
    force=False
)

t = rts.features.text.find_locations(transcript)

INFO:RTS:Loading spacy model: fr_core_news_lg


In [13]:
clips = rts.pipeline.extract_location_clips_from_transcript(transcript,
    25,
    OUTDIR,
    video_path,
    min_seconds=3,
    extend_duration=6,
    num_images=3,
    force=True
)

In [66]:
res = rts.features.text.enrich_scenes_with_transcript(clips.copy(), t)
# res

185.96 191.96  --  172.72 186.4
{'t': 'Demain, le personnel rencontrera à Berne la task force chargée de la future Crocer.', 's': '185.96', 'e': '191.96', 'locations': 'Bern'}
351.96 354.96  --  361.88 373.56
{'t': "à Bellinzone, le chauffeur turc qui est à l'origine", 's': '351.96', 'e': '354.96', 'locations': 'Bellinzona'}
1091.36 1094.36  --  1081.08 1090.88
1091.36 1094.36  --  1103.48 1110.24
{'t': 'à Genève, Bern, Baal et Saint-Gal.', 's': '1091.36', 'e': '1094.36', 'locations': 'Bern | Genève | Basel | Sankt Gallen'}


In [51]:
swiss_cities = rts.features.text.get_swiss_cities()

In [26]:
input_video_folder = os.path.join(REMOTE_VIDEOS, '0', '0', '0', 'ZB001000')
# input_video_folder = os.path.join(VIDEOS, '0', '0', 'ZB002000')
# v, a = rts.io.archive.get_raw_video_audio_parts(input_video_folder)

# '/'.join(input_video_folder.split('/')[-4:])
# remuxed = rts.pipeline.create_optimized_media(input_video_folder, LOCAL_VIDEOS, force=True)
# print(remuxed)

rts.pipeline.process_media(input_video_folder, LOCAL_VIDEOS, 
    compute_transcript=False, force_scene=False, force_trans=False)

ERROR:RTS:input length is 0: line 1 column 1 (char 0)
ERROR:RTS:Video file not found.
ERROR:RTS:Type is not JSON serializable: FrameTimecode


{'mediaId': 'ZB001000', 'status': 'fail', 'error': 'Could not extract scenes'}

In [ ]:
%ls {remux_folder}

In [ ]:
fts.groupby('mediaId').count().sort_values('location', ascending=False)

In [ ]:
fts

In [ ]:
import weaviate

client = weaviate.Client("http://localhost:8080") # or another location where your Weaviate instance is running

client.schema.get() # get the full schema as example

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
swiss_cities['Genève']

In [ ]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))

In [ ]:
import av
import av.datasets


video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)
out_path = os.path.join(OUTDIR, 'thumbs')

content = v

print(content)

with av.open(content) as container:
    # Signal that we only want to look at keyframes.
    stream = container.streams.video[0]
    stream.codec_context.skip_frame = "NONKEY"

    # print(stream.codec_context.height)
    for i, frame in enumerate(container.decode(stream)):
        if i % 4 != 0:
            continue


    #     print(frame)
    #     print(frame.time, frame.time_base)
        # We use `frame.pts` as `frame.index` won't make must sense with the `skip_frame`.
        frame.to_image().save(
            os.path.join(out_path, f"f{frame.pts:04d}.jpg"),
            quality=80,
        )

In [ ]:
thumbs_dir = os.path.join(OUTDIR, 'thumbs')
outfast = './notebook/fastdup'
fastdup_data = './notebook/out'

In [ ]:
import fastdup

In [ ]:

fastdup.run(input_dir=thumbs_dir, work_dir=fastdup_data, nearest_neighbors_k=8, turi_param='ccthreshold=0.90')    #main running function.
# fastdup.create_duplicates_gallery('out/similarity.csv', save_path=outfast)     #create a visual gallery of found duplicates
# fastdup.create_outliers_gallery('out/outliers.csv',   save_path=outfast)       #create a visual gallery of anomalies
# fastdup.create_components_gallery(fastdup_data, save_path=outfast)                    #create visualiaiton of connected components
# fastdup.create_stats_gallery('out', save_path=outfast, metric='blur')          #create visualization of images stastics (for example blur)
# fastdup.create_similarity_gallery('out', save_path=outfast,get_label_func=lambda x: x.split('/')[-2])     #create visualization of top_k similar images assuming data have labels which are in the folder name
# fastdup.create_aspect_ratio_gallery('out', save_path=outfast)

In [ ]:
config = fastdup.load_config(fastdup_data)
df = fastdup.galleries.do_find_top_components(work_dir=fastdup_data, input_dir=config['input_dir'], kwargs={})